# Download the images to a local directory

NOTE: this only needs to be done once for a given set of images. Once they are loaded into the bucket it doesn't need to be run again.

This code uses a list of accession numbers (found as a column in a CSV file) to generate IIIF Image API (v2) URLs for JPEG images that are 1000 pixels in the shortest dimension, then download them into a local directory.

After generating and downloading the images, they need to be uploaded to the Google Cloud bucket used in the Vision analysis.

In [77]:
import pandas as pd
import requests
import shutil # high-level file operations

# Load the image data into a dataframe
base_path = '/Users/baskausj/github/vandycite/gallery_buchanan/image_analysis/'
download_path = '/Users/baskausj/Downloads/'

# Load the source image data into a dataframe
source_image_dataframe = pd.read_csv(base_path + 'combined_images.csv', dtype=str)
# Set the commons_id column as the index
source_image_dataframe = source_image_dataframe.set_index('commons_id')

source_image_dataframe.head()

,local_filename,qid,accession_number,rank,kilobytes,height,width,photo_inception,extension,directory,label_en,commons_image_name,iiif_manifest,manifest_label,upload_notes
commons_id,,,,,,,,,,,,,,,
M122562148,1956-001.tif,Q102961253,1956.001,primary,73531,2611,4805,2020-07-23,tif,1956,A group of muffs and other articles of dress o...,A group of muffs and other articles of dress o...,https://iiif-manifest.library.vanderbilt.edu/g...,NaN,NaN
M122617251,1956.002.tif,Q103296446,1956.002,primary,86058,4471,3284,2012-10-17,tif,1956,A Flower Piece (after Jan van Huysum),A Flower Piece (after Jan van Huysum) - Vander...,https://iiif-manifest.library.vanderbilt.edu/g...,NaN,NaN
M122641532,1956-003.tif,Q103297456,1956.003,primary,47806,3600,2265,2020-08-18,tif,1956,Bishop Hacket,Bishop Hacket - Vanderbilt Fine Arts Gallery -...,https://iiif-manifest.library.vanderbilt.edu/g...,NaN,NaN
M122695514,1956.004.jpg,Q103310070,1956.004,primary,122,696,452,2010-02-22,jpg,1956,The Raising of Lazarus (after Leandro Bassano),The Raising of Lazarus (after Leandro Bassano)...,https://iiif-manifest.library.vanderbilt.edu/g...,NaN,NaN
M122611522,1956.006.jpg,Q102974173,1956.006,primary,58,365,436,2010-02-22,jpg,1956,The Farmhouse by the Water,The Farmhouse by the Water - Vanderbilt Fine A...,https://iiif-manifest.library.vanderbilt.edu/g...,NaN,NaN


In [78]:

# Import CSV data as a dataframe.
accession_dataframe = pd.read_csv(base_path + 'test_accession_numbers.csv', dtype=str)

# Loop through the dataframe rows and download the images.
for index, row in accession_dataframe.iterrows():
    accession_number = row['accession_number']
    print(accession_number)

    # Look up the image data in the source image dataframe.
    # In cases where there are two images, we want the primary image.
    image_series = source_image_dataframe.loc[(source_image_dataframe['accession_number'] == accession_number) & (source_image_dataframe['rank'] == 'primary')]
    manifest_url = image_series['iiif_manifest'][0]

    # get the manifest from the manifest url
    manifest = requests.get(manifest_url).json()
    #print(json.dumps(manifest, indent=2))
    service_url = manifest['sequences'][0]['canvases'][0]['images'][0]['resource']['service']['@id']
    # Because of the error in original manifests, replace version 3 with version 2 in the URL.
    service_url = service_url.replace('/3/', '/2/')
    #print('service_url', service_url)

    # Determine the maximum and minimum dimensions of the image.
    height = image_series['height'][0]
    #print('height', height)
    width = image_series['width'][0]
    #print('width', width)
    shortest_dimension = min(int(height), int(width))
    longest_dimension = max(int(height), int(width))
    #print('shortest_dimension', shortest_dimension)

    # We want to know what the largest dimension needs to be for the shortest dimension to be 1000 pixels.
    # If that calculation makes the longest dimension longer than the actual longest dimension, 
    # then we want to use the actual longest dimension.
    # If the shortest dimension is already less than 1000 pixels, then we will just use the longest dimension as is.
    if shortest_dimension > 1000:
        size = int(1000 * (longest_dimension / shortest_dimension))
        if size > longest_dimension:
            size = longest_dimension
    else:
        size = longest_dimension
    #print('size', size)

    # construct the image url using the "!" size option, that keeps the aspect ratio but sizes to the maximum dimension.
    image_url = service_url + '/full/!' + str(size) + ',' + str(size) + '/0/default.jpg'
    print('image_url', image_url)
    print()
        
    # retrieve the image from the IIIF server
    image_object = requests.get(image_url, stream=True).raw

    # save the image as a JPEG file]
    with open(download_path + 'google_vision_images/' + accession_number + '.jpg', 'wb') as out_file:
        shutil.copyfileobj(image_object, out_file)

print('done')

1986.076
image_url https://iiif.library.vanderbilt.edu/iiif/2/gallery%2F1986%2F1986.076.tif/full/!1339,1339/0/default.jpg

1979.0324P
image_url https://iiif.library.vanderbilt.edu/iiif/2/gallery%2F1979%2F1979.0324P.tif/full/!1217,1217/0/default.jpg

done


# Google Cloud Vision image analysis

The first cell retrieves the service key, creates a credentials object, then uses it to authenticate and create a `client` object.

In [3]:
# Here's the landing page for Google Cloud Vision
# https://cloud.google.com/vision/
# From it you can try the api by dragging and dropping an image into the browser. You can then 
# view the JSON response, which was helpfule at first to understand the structure of the response.

# The following tutorial contains critical information about enabling the API and creating a role
# for the service account to allow it access. This is followed by creating a service account key.
# https://cloud.google.com/vision/docs/detect-labels-image-client-libraries

# I didn't actually do this tutorial, but it was useful to understand the order of operations that
# needed to be done prior to writing to the API.
# https://www.cloudskillsboost.google/focuses/2457?parent=catalog&utm_source=vision&utm_campaign=cloudapi&utm_medium=webpage
# Because I'm using the Python client library, the part about setting up the request body was irrelevant. 
# But the stuff about uploading the files to the bucket, making it publicly accessible, etc. was helpful.
import json
import pandas as pd

# Imports the Google Cloud client library
# Reference for Google Cloud Vision Python client https://cloud.google.com/python/docs/reference/vision/latest
from google.cloud import vision
from google.cloud import vision_v1
from google.cloud.vision_v1 import AnnotateImageResponse

# Import from Google oauth library
from google.oauth2 import service_account

# Customize for your own computer
user_dir = 'baskausj' # Enter your user directory name here
base_path = '/Users/baskausj/github/vandycite/gallery_buchanan/image_analysis/' # Location of the accession number data file

# Set the path to the service account key
key_path = '/Users/' + user_dir + '/image-analysis-376619-193859a33600.json'

# Create a credentials object from the service account key
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# API documentation https://cloud.google.com/python/docs/reference/vision/latest/google.cloud.vision_v1.services.image_annotator.ImageAnnotatorClient#methods
# The first two versions have no arguments and the credentials are loaded from the environment variable.
#client = vision.ImageAnnotatorClient()
# Used this specific v1 to get the JSON conversion to work
#client = vision_v1.ImageAnnotatorClient()
# Use this line instead of the one above to load the credentials directly from the file
client = vision_v1.ImageAnnotatorClient(credentials=credentials)


Load the source data from a CSV. The critical column needed here is the `accession_number` column, since it is the one that was used to construct the image file name for the uploaded test images.

In [11]:
# Import CSV data as a dataframe.
accession_dataframe = pd.read_csv(base_path + 'test_accession_numbers.csv', dtype=str)
accession_dataframe.head()

,QID,accession_number,Artwork Name,Artist,Category,Notes
0,Q103312093,1979.0322P,Virgin,Unknown,"Painting, religious","Woman, Virgin Mary, angel/cherub"
1,Q102974452,1971.002,The Jungfrau,Edward Theodore Compton,"Painting, landscape","Mountains, rocks, trees, animal"
2,Q102969583,1979.0016,Paris Garden,E. Boyd-Smith,"Painting, landscape","Trees, gate"
3,Q103306344,1979.0249P,Parlor Scene,Hans Buchner,"Painting, portrait","Man, woman, painting, bench/table, animal rug"
4,Q103299404,1979.0236P,Dutch Interior,Louis-Simon Cabaillot-Lassalle,"Painting, portrait","Woman, maid, plant, bird"


In [12]:
# This cell is for testing the API with a single image
# Don't run this cell if you want to run the whole dataframe
accession_dataframe = accession_dataframe.head(1)

Loop through all of the accession numbers and perfomr the analysis on each of the images.

In [15]:
# Loop through the dataframe rows and download the images.
for index, row in accession_dataframe.iterrows():
    accession_number = row['accession_number']
    print('accession_number', accession_number)

    # To access the images, they should be stored in a Google Cloud Storage bucket that is set up for public access.
    # It's also possible to use a publicly accessible URL, but that seems to be unreliable.
    # The storage costs for a few images are negligible.

    # Construct the path to the image file
    image_uri = 'gs://vu-gallery/' + accession_number + '.jpg'
    print('image_uri', image_uri)

    # Here is the API documentation for the Feature object.
    # https://cloud.google.com/vision/docs/reference/rest/v1/Feature
    #analysis_type = vision.Feature.Type.FACE_DETECTION
    #analysis_type = vision.Feature.Type.LABEL_DETECTION
    analysis_type = vision.Feature.Type.OBJECT_LOCALIZATION

    # This API documentation isn't exactly the one for the .annotate_image method, but it's close enough.
    # https://cloud.google.com/vision/docs/reference/rest/v1/projects.images/annotate
    # In particular, it links to the AnnotateImageRequest object, which is what we need to pass to the annotate_image method.
    response = client.annotate_image({
    'image': {'source': {'image_uri': image_uri}},
    'features': [{'type_': analysis_type}]
    })

    # The API response is a protobuf object, which is not JSON serializable.
    # So we need to convert it to a JSON serializable object.
    # Solution from https://stackoverflow.com/a/65728119
    response_json = AnnotateImageResponse.to_json(response)

    # The structure of the response is detailed in the API documentation here:
    # https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse
    # The various bits are detailed for each feature type.
    # Here's the documentation for entity annotations, with a link to the BoundyPoly object.
    # https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse#EntityAnnotation
    response_struct = json.loads(response_json)
    print(response_json)

accession_number 1979.0322P
image_uri gs://vu-gallery/1979.0322P.jpg
{
  "localizedObjectAnnotations": [
    {
      "mid": "/m/01g317",
      "name": "Person",
      "score": 0.59910905,
      "boundingPoly": {
        "normalizedVertices": [
          {
            "x": 0.7183214,
            "y": 0.6143456
          },
          {
            "x": 0.9937997,
            "y": 0.6143456
          },
          {
            "x": 0.9937997,
            "y": 0.8886199
          },
          {
            "x": 0.7183214,
            "y": 0.8886199
          }
        ],
        "vertices": []
      },
      "languageCode": ""
    },
    {
      "mid": "/m/01g317",
      "name": "Person",
      "score": 0.5477003,
      "boundingPoly": {
        "normalizedVertices": [
          {
            "x": 0.031403385,
            "y": 0.018900577
          },
          {
            "x": 0.7862895,
            "y": 0.018900577
          },
          {
            "x": 0.7862895,
            "y": 0